<a href="https://colab.research.google.com/github/Pamela2710/Data-Mining/blob/main/TextMining/TextMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
#df = pd.read_csv("sample_data/livetweets_data_no_json.csv",encoding='latin1', skipinitialspace=True)
#df.head()
pd.read_csv("sample_data/livetweets_data_no_json.csv",encoding='latin1', skipinitialspace=True)


<ipython-input-3-c6b34cba91d4>:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("sample_data/livetweets_data_no_json.csv",encoding='latin1', skipinitialspace=True)


,tweet_id,tweet_date,tweet_screen_name,tweet_text
0,1096977737930158080,2019-02-17,gabrielguerronv,@AlexTonello5RC @CompromisoRC5 @PaolaPabonC @M...
1,1096977884848082944,2019-02-17,jhonav00,@PaolaPabonC @CompromisoRC5 @MashiRafael @Alex...
2,1096981630881882113,2019-02-17,Cristhyn3000,Que importante ha sido sentir el calor de los ...
3,1096982892759130112,2019-02-17,PelotonFusilero,Mi campaÃ±a no tiene pactos polÃ­ticos ni apor...
4,1096983080072556544,2019-02-17,carosu_92,Â¡Lo mÃ¡s avergonzarte que he visto cÃ³mo muje...
...,...,...,...,...
738859,1108940225282113537,2019-03-22,andrescuevaj,Guayaquil tiene un modelo. Un modelo que traba...
738860,1108940350389776384,2019-03-22,mlpatinol,@coshito4977 @PacoMoncayo @notihoyecuador Los ...
738861,1108940356245028865,2019-03-22,LEOVIJAR,@nessawoolf86 @radiosucesosec @MashiRafael @Lo...
738862,1108940365908725762,2019-03-22,Patrici02306665,"@CynthiaViteri6 Gigante caravana, y una cantid..."


In [ ]:
df = pd.read_csv("sample_data/livetweets_data_no_json.csv")
#df.head()

<ipython-input-4-f508dc6e459c>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("sample_data/livetweets_data_no_json.csv")


In [ ]:
df_candidatos = pd.read_csv("sample_data/account_info.csv")


In [ ]:
df_candidatos

,id,nombre,apellidos,ciudad,tipo,twitter_id,twitter_screen_name,twitter_date
0,1,Walter Javier,Gómez Ronquillo,País,S,718198950432530432,wgomezr,2016-04-07
1,2,Teddy Gustavo,Tama Aguirre,País,S,1094654440823578626,TamaApoyo,2019-02-10
2,3,Carlos Gonzalo,Mosquera Benalcázar,País,S,1079154149160374274,MosqueraCarlosB,2018-12-29
3,4,Carlos Eduardo,Figueroa Figueroa,País,S,263235527,carlosfigueroa6,2011-03-09
4,5,José Carlos,Tuarez Zambrano,País,S,1048006557907275777,jcarlostuarez,2018-10-05
5,6,Aland Javier,Molestina Malta,País,S,755944679422750724,ALANDMOLESTINAM,2016-07-21
6,7,Christian Antonio,Cruz Larrea,País,S,1088221704084705280,ChristianCruzLa,2019-01-23
7,8,David Alejandro,Rosero Minda,País,S,281302673,davidroserow,2011-04-13
8,9,Pablo Eduardo,Ponce Cerda,País,S,174037261,PabloPonce,2010-08-02
9,10,Alfredo Humberto,Alvear Báez,País,S,1095886490385764352,CPCCS2019,2019-02-14


In [ ]:
df

,tweet_id,tweet_date,tweet_screen_name,tweet_text
0,1096977737930158080,2019-02-17,gabrielguerronv,@AlexTonello5RC @CompromisoRC5 @PaolaPabonC @M...
1,1096977884848082944,2019-02-17,jhonav00,@PaolaPabonC @CompromisoRC5 @MashiRafael @Alex...
2,1096981630881882113,2019-02-17,Cristhyn3000,Que importante ha sido sentir el calor de los ...
3,1096982892759130112,2019-02-17,PelotonFusilero,Mi campaña no tiene pactos políticos ni aporte...
4,1096983080072556544,2019-02-17,carosu_92,¡Lo más avergonzarte que he visto cómo mujer! ...
...,...,...,...,...
747267,1109079427881816064,2019-03-22,Sebasgarcia30,En #NotiMundoEstelar conversamos con @PacoMonc...
747268,1109079630298927105,2019-03-22,jimmygavilanes4,¡Toma tu maduro!\n\nUn revés predecible para e...
747269,1109079429706334208,2019-03-22,HolmerParraga,@davalos2019 Dale
747270,1109079433405702144,2019-03-22,transsar1,"Cerramos la campaña con alegría , con propuest..."


In [ ]:
df_tweets = df.fillna("")
df_tweets

,tweet_id,tweet_date,tweet_screen_name,tweet_text
0,1096977737930158080,2019-02-17,gabrielguerronv,@AlexTonello5RC @CompromisoRC5 @PaolaPabonC @M...
1,1096977884848082944,2019-02-17,jhonav00,@PaolaPabonC @CompromisoRC5 @MashiRafael @Alex...
2,1096981630881882113,2019-02-17,Cristhyn3000,Que importante ha sido sentir el calor de los ...
3,1096982892759130112,2019-02-17,PelotonFusilero,Mi campaña no tiene pactos políticos ni aporte...
4,1096983080072556544,2019-02-17,carosu_92,¡Lo más avergonzarte que he visto cómo mujer! ...
...,...,...,...,...
747267,1109079427881816064,2019-03-22,Sebasgarcia30,En #NotiMundoEstelar conversamos con @PacoMonc...
747268,1109079630298927105,2019-03-22,jimmygavilanes4,¡Toma tu maduro!\n\nUn revés predecible para e...
747269,1109079429706334208,2019-03-22,HolmerParraga,@davalos2019 Dale
747270,1109079433405702144,2019-03-22,transsar1,"Cerramos la campaña con alegría , con propuest..."


In [ ]:
# Filtra la tabla de candidatos solo para aquellos cuyos twitter_screen_name aparecen en la tabla de tweets
candidatos_filtrados = df_tweets[df_tweets['tweet_screen_name'].isin(df_candidatos['twitter_screen_name'])]

# Obtén la frecuencia de cada tweet_screen_name en la tabla de tweets
frecuencia_tweet_screen_name = candidatos_filtrados['tweet_screen_name'].value_counts()

# Muestra el resultado
top_10 = frecuencia_tweet_screen_name.head(10)


# Muestra el resultado
print(top_10)

HernanUlloa        730
jimmyjairala       519
abenavidesgol1     401
LuisaMaldonadoM    393
VickyDesintonio    329
fcojimenez21       306
LoroHomero         287
mrivadeneirac      272
CynthiaViteri6     241
wgomezr            232
Name: tweet_screen_name, dtype: int64


In [100]:
#Tabla del Top 10

# Filtra la tabla de candidatos_filtrados solo para aquellos en el top 10
top_10_candidatos_filtrados = candidatos_filtrados[candidatos_filtrados['tweet_screen_name'].isin(top_10.index)]

top_10_candidatos_filtrados

#Tabla del Top 10

# Filtra la tabla de candidatos_filtrados solo para aquellos en el top 10
top_10_candidatos_filtrados2 = candidatos_filtrados[candidatos_filtrados['tweet_screen_name'].isin(top_10.index)]

top_10_candidatos_filtrados2


,tweet_id,tweet_date,tweet_screen_name,tweet_text
21,1096984892318015489,2019-02-17,abenavidesgol1,Énner Valencia estuvo presente en las redes co...
152,1097006152498532353,2019-02-17,abenavidesgol1,Jairo Vélez cumplió destacada actuación en tri...
158,1097008003398672384,2019-02-17,LuisaMaldonadoM,Desde cada uno de los rincones de #Quito los q...
195,1097012227855564800,2019-02-17,abenavidesgol1,Pocos minutos en cancha estuvo Fernando Gaibor...
1157,1097162552348418050,2019-02-17,LoroHomero,Disfrutamos una linda noche en Barrio #ElConde...
...,...,...,...,...
71082,1100006823753641984,2019-02-25,mrivadeneirac,Incentivar la participación ciudadana desde la...
71177,1100009392500559873,2019-02-25,mrivadeneirac,[ENTREVISTA] El desconocimiento del Quito Pode...
71311,1100014105954529280,2019-02-25,CynthiaViteri6,¡Excelente inicio de semana para todos los gua...
71391,1100017097361502208,2019-02-25,CynthiaViteri6,Estuvimos este fin de semana en la 8va brigada...


In [ ]:
agrupado_por_usuario = top_10_candidatos_filtrados.groupby('tweet_screen_name')['tweet_text'].apply(' '.join).reset_index(name='tweets_agrupados')
agrupado_por_usuario

,tweet_screen_name,tweets_agrupados
0,CynthiaViteri6,¡Buenos días guayaquileños! Ya estamos en vivo...
1,HernanUlloa,@HernanUlloa abogado se ganó mi respeto y admi...
2,LoroHomero,Disfrutamos una linda noche en Barrio #ElConde...
3,LuisaMaldonadoM,Desde cada uno de los rincones de #Quito los q...
4,VickyDesintonio,#ManabíEsVictoria\nAmigas y amigos deseo para ...
5,abenavidesgol1,Énner Valencia estuvo presente en las redes co...
6,fcojimenez21,¡Seguimos recorriendo Guayaquil! Estamos en Me...
7,jimmyjairala,Se ve que los adversarios políticos de @jimmyj...
8,mrivadeneirac,En pocos minutos daré inicio a mi día con @tvm...
9,wgomezr,Comparto con ustedes mí Agenda de actividades ...


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [41]:
# Preprocesamiento de texto
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('spanish'))  # Obtener stopwords en español
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenización y conversión a minúsculas
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in stop_words]  # Eliminar stopwords y signos de puntuación
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]  # Lematización
    #processed_text = ' '.join(lemmatized_tokens)  # Unir los tokens en una cadena de texto
    print(lemmatized_tokens)
    return lemmatized_tokens

# Aplicar preprocesamiento de texto a los tweets
agrupado_por_usuario['preprocessed_text'] = agrupado_por_usuario['tweets_agrupados'].apply(preprocess_text)
#agrupado_por_usuario['preprocessed_text'] = agrupado_por_usuario['tweets_agrupados'].apply(clean_text)
agrupado_por_usuario

['días', 'guayaquileños', 'vivo', 'entrevista', 'engrandecer', 'aún', 'ciudad', 'http', 'siempre', 'vamos', 'ir', 'allá', 'competencias', 'demostrado', 'seguiremos', 'haciendo', 'respecta', 'educación', 'seguridad', 'salud', 'bienestar', 'familias', 'guayaquileñas', 'http', 'nueva', 'aerovía', 'bus', 'eléctricos', 'puentes', 'samborondón', 'daule', 'bicicletas', 'públicas', 'vamos', 'seguir', 'dando', 'mayor', 'facilidades', 'conductores', 'pasajeros', 'lleguen', 'tiempo', 'destinos', 'soñar', 'grande', 'http', 'sé', 'perfectamente', 'sienten', 'madres', 'envían', 'hijos', 'escuelas', 'implementaremos', 'cámaras', 'seguridad', 'afuera', 'colegios', 'escuelas', 'alejar', 'vendedores', 'drogas', 'hijos', 'http', 'jóvenes', 'cantera', 'acompañaron', 'candidata', 'cynthiaviteri6', 'gran', 'caravana', 'distrito2', 'junto', 'candidatos', 'josuesanchezec', 'luzmilanicolald', 'caicedoegis', 'jxcabrer', 'futuro', 'prefecto', 'clmoralesb', 'campaña', 'http', 'sergio', 'toral', 'progreso', 'guaya

,tweet_screen_name,tweets_agrupados,preprocessed_text
0,CynthiaViteri6,¡Buenos días guayaquileños! Ya estamos en vivo...,"[días, guayaquileños, vivo, entrevista, engran..."
1,HernanUlloa,@HernanUlloa abogado se ganó mi respeto y admi...,"[hernanulloa, abogado, ganó, respeto, admiraci..."
2,LoroHomero,Disfrutamos una linda noche en Barrio #ElConde...,"[disfrutamos, linda, noche, barrio, elconde4, ..."
3,LuisaMaldonadoM,Desde cada uno de los rincones de #Quito los q...,"[cada, rincones, quito, quiteños, quiteñas, mu..."
4,abenavidesgol1,Énner Valencia estuvo presente en las redes co...,"[énner, valencia, presente, redes, contrarias,..."
5,c_espinozaec,La reconstrucción de Manabí será uno de los pi...,"[reconstrucción, manabí, pilares, trabajo, con..."
6,fcojimenez21,¡Seguimos recorriendo Guayaquil! Estamos en Me...,"[recorriendo, guayaquil, mercado, santa, teres..."
7,jimmyjairala,Se ve que los adversarios políticos de @jimmyj...,"[ve, adversarios, políticos, jimmyjairala, baj..."
8,mrivadeneirac,En pocos minutos daré inicio a mi día con @tvm...,"[pocos, minutos, daré, inicio, día, tvmanabita..."
9,wgomezr,Comparto con ustedes mí Agenda de actividades ...,"[comparto, ustedes, agenda, actividades, cañar..."


In [101]:
# Aplicar preprocesamiento de texto a los tweets
top_10_candidatos_filtrados2['preprocessed_text'] = top_10_candidatos_filtrados2['tweet_text'].apply(preprocess_text)
#agrupado_por_usuario['preprocessed_text'] = agrupado_por_usuario['tweets_agrupados'].apply(clean_text)
top_10_candidatos_filtrados2

['énner', 'valencia', 'presente', 'redes', 'contrarias', 'triunfo', 'tigres', 'necaxa', 'http', 'http']
['jairo', 'vélez', 'cumplió', 'destacada', 'actuación', 'triunfo', 'atlante', 'dorado', 'http', 'http']
['cada', 'rincones', 'quito', 'quiteños', 'quiteñas', 'muestran', 'apoyo', 'cariño', 'militante', 'revolución', 'ciudadana', 'luisamaldonadom', 'sur', 'lista', '5', 'compromiso', 'social', 'arecuperarlapatria', 'todotodito5', 'http']
['pocos', 'minutos', 'cancha', 'fernando', 'gaibor', 'empate', 'independiente', 'avellaneda', 'http', 'http']
['disfrutamos', 'linda', 'noche', 'barrio', 'elconde4', 'gracias', 'recibimiento', 'tan', 'emotivo', 'permitirnos', 'compartir', 'plan', 'trabajo', 'lorohomero', 'futuro', 'alcalde', 'quito', 'votatodo19', 'quitograndeotravez', 'http']
['negado', 'recurso', 'apelación', 'propuesto', 'barcelona', 'nivel', 'conmebol', 'http', 'http']
['felipe', 'caicedo', 'actuó', 'cambio', 'derrota', 'sufrida', 'lazio', 'http', 'http']
['independiente', 'valle',

<ipython-input-101-72aa7e21e5d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_10_candidatos_filtrados2['preprocessed_text'] = top_10_candidatos_filtrados2['tweet_text'].apply(preprocess_text)


,tweet_id,tweet_date,tweet_screen_name,tweet_text,preprocessed_text
21,1096984892318015489,2019-02-17,abenavidesgol1,Énner Valencia estuvo presente en las redes co...,"[énner, valencia, presente, redes, contrarias,..."
152,1097006152498532353,2019-02-17,abenavidesgol1,Jairo Vélez cumplió destacada actuación en tri...,"[jairo, vélez, cumplió, destacada, actuación, ..."
158,1097008003398672384,2019-02-17,LuisaMaldonadoM,Desde cada uno de los rincones de #Quito los q...,"[cada, rincones, quito, quiteños, quiteñas, mu..."
195,1097012227855564800,2019-02-17,abenavidesgol1,Pocos minutos en cancha estuvo Fernando Gaibor...,"[pocos, minutos, cancha, fernando, gaibor, emp..."
1157,1097162552348418050,2019-02-17,LoroHomero,Disfrutamos una linda noche en Barrio #ElConde...,"[disfrutamos, linda, noche, barrio, elconde4, ..."
...,...,...,...,...,...
71082,1100006823753641984,2019-02-25,mrivadeneirac,Incentivar la participación ciudadana desde la...,"[incentivar, participación, ciudadana, educaci..."
71177,1100009392500559873,2019-02-25,mrivadeneirac,[ENTREVISTA] El desconocimiento del Quito Pode...,"[entrevista, desconocimiento, quito, poder, ll..."
71311,1100014105954529280,2019-02-25,CynthiaViteri6,¡Excelente inicio de semana para todos los gua...,"[inicio, semana, guayaquileños, empezamos, lun..."
71391,1100017097361502208,2019-02-25,CynthiaViteri6,Estuvimos este fin de semana en la 8va brigada...,"[fin, semana, 8va, brigada, desintoxicación, j..."


In [95]:
# Aplicar preprocesamiento de texto a los tweets
df_tweets['preprocessed_text'] = df_tweets['tweet_text'].apply(preprocess_text)
#agrupado_por_usuario['preprocessed_text'] = agrupado_por_usuario['tweets_agrupados'].apply(clean_text)
df_tweets

Se truncaron las últimas líneas 5000 del resultado de transmisión.
['luisamaldonadom', 'vamos', 'vencer', 'voto', 'vende']
['jimmyjairala', 'ratifica', 'compromiso', 'lograr', 'elmejorguayaquildetodoslostiempos', 'cooperativas', 'guasmosur', 'gran', 'caravana', 'motorizada', 'llegó', '5', 'esquinas', 'presidió', 'concentración', 'moradores', 'sector', 'unieron', 'proyecto', 'jairalista', 'http']
['gracias', 'compañeros', 'ecuatoriana', 'invitarnos', 'evento', 'honor', 'ser', 'mama', 'carnaval', 'recibir', 'bastón', 'mando', 'hacemos', 'mucha', 'humildad', 'compromiso', 'inclaudicable', 'siempre', 'ustedes', 'interculturalidad', 'votatodo5', 'http']
['gracias', 'recibirnos', 'compañeros', 'barrio', '4', 'abril', 'dirigente', 'sector', 'popular', 'comprendo', 'necesidad', 'preocupación', 'alcaldía', 'seguiremos', 'apoyando', 'regularización', 'barrio', 'escrituras', 'elmomentoesahora', 'votatodo5', 'http']
['sr', 'cpccst', 'si', 'podemos', 'sumar', 'mentalmente', 'utilicemos', 'calculado

,tweet_id,tweet_date,tweet_screen_name,tweet_text,preprocessed_text
0,1096977737930158080,2019-02-17,gabrielguerronv,@AlexTonello5RC @CompromisoRC5 @PaolaPabonC @M...,"[alextonello5rc, compromisorc5, paolapabonc, m..."
1,1096977884848082944,2019-02-17,jhonav00,@PaolaPabonC @CompromisoRC5 @MashiRafael @Alex...,"[paolapabonc, compromisorc5, mashirafael, alex..."
2,1096981630881882113,2019-02-17,Cristhyn3000,Que importante ha sido sentir el calor de los ...,"[importante, sido, sentir, calor, habitantes, ..."
3,1096982892759130112,2019-02-17,PelotonFusilero,Mi campaña no tiene pactos políticos ni aporte...,"[campaña, pactos, políticos, aportes, millonar..."
4,1096983080072556544,2019-02-17,carosu_92,¡Lo más avergonzarte que he visto cómo mujer! ...,"[avergonzarte, visto, cómo, mujer, siento, ide..."
...,...,...,...,...,...
71481,1100019635179716608,2019-02-25,CarmenLara67,@PaolaPabonC @Elimpactoinform @CapiZapataEC @E...,"[paolapabonc, elimpactoinform, capizapataec, e..."
71482,1100019767371591681,2019-02-25,MartinG5425290,@VickyDesintonio En ustedes confiamos pueblo h...,"[vickydesintonio, ustedes, confiamos, pueblo, ..."
71483,1100019849189908481,2019-02-25,Jrojasm45,En todas las paradas del metro deberán haber e...,"[toda, paradas, metro, deberán, haber, dispens..."
71484,1100019903237640194,2019-02-25,yovamajo,por favor NO VOTAR BLANCO O NULO... eso sólo b...,"[favor, votar, blanco, nulo, sólo, beneficiarí..."


In [54]:
from collections import Counter

# Crear un diccionario para almacenar las frecuencias
frecuencias_por_usuario = {}

# Iterar sobre cada fila del DataFrame
for index, row in agrupado_por_usuario.iterrows():
    tweet_screen_name = row['tweet_screen_name']
    preprocessed_text = row['preprocessed_text']

    # Contar las frecuencias de palabras en el texto preprocesado
    frecuencias_palabras = Counter(preprocessed_text).most_common(10)

    # Almacenar las frecuencias en el diccionario, utilizando tweet_screen_name como clave
    frecuencias_por_usuario.setdefault(tweet_screen_name, []).append(frecuencias_palabras)
#print(frecuencias_por_usuario)
# Crear un DataFrame a partir del diccionario de frecuencias
df_frecuencias = pd.DataFrame(frecuencias_por_usuario).transpose().fillna(0)
df_frecuencias

,0
CynthiaViteri6,"[(http, 37), (guayaquil, 17), (jóvenes, 12), (..."
HernanUlloa,"[(http, 33), (hernanulloa, 26), (cpccs, 25), (..."
LoroHomero,"[(lorohomero, 30), (http, 23), (quito, 9), (al..."
LuisaMaldonadoM,"[(http, 64), (votatodo5, 41), (elmomentoesahor..."
abenavidesgol1,"[(http, 191), (triunfo, 21), (abenavidesgol1, ..."
c_espinozaec,"[(http, 41), (espinozaescomoyo, 29), (turector..."
fcojimenez21,"[(http, 56), (jiménezalcalde, 29), (elguayaqui..."
jimmyjairala,"[(http, 88), (jimmyjairala, 40), (elmejorguaya..."
mrivadeneirac,"[(http, 83), (ecuador, 60), (cpccs, 59), (mari..."
wgomezr,"[(http, 38), (cpccs, 28), (ecuador, 17), (walt..."


In [107]:
import math
# Paso 4: Concatenar todos los tweets de los top 10 candidatos en una sola cadena
texto_campaña = ' '.join(top_10_candidatos_filtrados['tweet_text'])
print('len ', len(top_10_candidatos_filtrados2))

# Paso 5: Preprocesar el texto (tokenizar, eliminar stopwords, lematizar, etc.)
# (Asumiendo que tienes una función preprocess_text() que hace esto)
texto_preprocesado = preprocess_text(texto_campaña)
print('texto_preprocesado ',texto_preprocesado)
print('text_procesado_len ', len(texto_preprocesado))
# Paso 6: Contar la frecuencia de palabras en el texto preprocesado
frecuencias_palabras = Counter(texto_preprocesado)
print('frecuencia ',frecuencias_palabras)

# Paso 7: Seleccionar las 10 palabras más comunes
top_10_palabras = frecuencias_palabras.most_common(10)
print('top 10 ',top_10_palabras)

print("\nTop 10 palabras más utilizadas en campaña por los top 10 candidatos:")

# Paso 8: Calcular los índices de términos: tf, df, idf, tf-idf
# Calcular TF para cada palabra
tf = {palabra: sqrt(frecuencia**2) frecuencia / len(top_10_candidatos_filtrados2) for palabra, frecuencia in top_10_palabras}
print('tf',tf)

# Calcular DF para cada palabra
claves = [tupla[0] for tupla in top_10_palabras]
print('key ',claves)
df = {palabra: sum(1 for tweet in top_10_candidatos_filtrados2['preprocessed_text'] if palabra in tweet) for palabra in claves}
print('df',df)

# Calcular IDF para cada palabra
num_tweets_top_candidatos = len(top_10_candidatos_filtrados)
idf = {palabra: math.log(num_tweets_top_candidatos / (df[palabra] + 1)) for palabra in claves}
print('idf',idf)
# Calcular TF-IDF para cada palabra
tfidf = {palabra: tf[palabra] * idf[palabra] for palabra in claves}

# Crear un DataFrame para mostrar los resultados
resultados = pd.DataFrame({
    'Palabra': list(claves),
    'TF': list(tf.values()),
    'DF': list(df.values()),
    'IDF': list(idf.values()),
    'TF-IDF': list(tfidf.values())
})

# Seleccionar las 10 palabras con el mayor TF-IDF
top_10_palabras_tfidf = resultados.sort_values(by='TF-IDF', ascending=False).head(10)

print("\nTabla con todos los índices de cálculo por términos: tf, df, idf, tf-idf")
print(top_10_palabras_tfidf)



len  591
['énner', 'valencia', 'presente', 'redes', 'contrarias', 'triunfo', 'tigres', 'necaxa', 'http', 'http', 'jairo', 'vélez', 'cumplió', 'destacada', 'actuación', 'triunfo', 'atlante', 'dorado', 'http', 'http', 'cada', 'rincones', 'quito', 'quiteños', 'quiteñas', 'muestran', 'apoyo', 'cariño', 'militante', 'revolución', 'ciudadana', 'luisamaldonadom', 'sur', 'lista', '5', 'compromiso', 'social', 'arecuperarlapatria', 'todotodito5', 'http', 'pocos', 'minutos', 'cancha', 'fernando', 'gaibor', 'empate', 'independiente', 'avellaneda', 'http', 'http', 'disfrutamos', 'linda', 'noche', 'barrio', 'elconde4', 'gracias', 'recibimiento', 'tan', 'emotivo', 'permitirnos', 'compartir', 'plan', 'trabajo', 'lorohomero', 'futuro', 'alcalde', 'quito', 'votatodo19', 'quitograndeotravez', 'http', 'negado', 'recurso', 'apelación', 'propuesto', 'barcelona', 'nivel', 'conmebol', 'http', 'http', 'felipe', 'caicedo', 'actuó', 'cambio', 'derrota', 'sufrida', 'lazio', 'http', 'http', 'independiente', 'valle

In [ ]:
#TF


calculos = pd.DataFrame()

In [47]:
# Crear un diccionario para almacenar las frecuencias
frecuencias_por_usuario = {}

# Iterar sobre cada fila del DataFrame
for index, row in agrupado_por_usuario.iterrows():
    tweet_screen_name = row['tweet_screen_name']
    preprocessed_text = row['preprocessed_text']

    # Contar las frecuencias de palabras en el texto preprocesado
    frecuencias_palabras = Counter(preprocessed_text)

    # Almacenar las frecuencias en el diccionario, utilizando tweet_screen_name como clave
    frecuencias_por_usuario[tweet_screen_name] = frecuencias_palabras

# Crear un DataFrame a partir del diccionario de frecuencias
df_frecuencias = pd.DataFrame(frecuencias_por_usuario).transpose().fillna(0)

# Obtener las 10 palabras más usadas por cada candidato
top_10_palabras_por_candidato = pd.DataFrame(index=df_frecuencias.index, columns=range(1, 11))

for tweet_screen_name in df_frecuencias.index:
    top_10_palabras = df_frecuencias.loc[tweet_screen_name].nlargest(10).index.tolist()
    top_10_palabras_por_candidato.loc[tweet_screen_name] = top_10_palabras

# Mostrar el resultado
print(top_10_palabras_por_candidato)

                         1                 2   \
CynthiaViteri6         http         guayaquil   
HernanUlloa            http       hernanulloa   
LoroHomero       lorohomero              http   
LuisaMaldonadoM        http         votatodo5   
abenavidesgol1         http           triunfo   
c_espinozaec           http  espinozaescomoyo   
fcojimenez21           http    jiménezalcalde   
jimmyjairala           http      jimmyjairala   
mrivadeneirac          http           ecuador   
wgomezr                http             cpccs   

                                                3   \
CynthiaViteri6                             jóvenes   
HernanUlloa                                  cpccs   
LoroHomero                                   quito   
LuisaMaldonadoM                   elmomentoesahora   
abenavidesgol1                      abenavidesgol1   
c_espinozaec                     turectorciudadano   
fcojimenez21               elguayaquilquetúquieres   
jimmyjairala     elmejorguay